# 日期

In [ ]:
DocTestSetup = :(using Dates)

`Dates` 模块提供了两种类型来处理日期：[`Date`](@ref) 和 [`DateTime`](@ref)，分别精确到日和毫秒；两者都是抽象类型 [`TimeType`](@ref) 的子类型。区分类型的动机很简单：不必处理更高精度所带来的复杂性时，一些操作在代码和思维推理上都更加简单。例如，由于 [`Date`](@ref) 类型仅精确到日（即没有时、分或秒），因此避免了时区、夏令时和闰秒等不必要的通常考虑。

[`Date`](@ref) 和 [`DateTime`](@ref) 类型都是基本不可变类型 [`Int64`](@ref) 的包装类。这两种类型的单个 `instant` 字段实际上属于 `UTInstant{P}` 类型。这种类型表示的是一种基于世界时间（UT）持续增长的机器时间 [^1]。[`DateTime`](@ref) 类型并不考虑时区（用 Python 的话讲，它是 *naive* 的），与 Java 8 中的 *LocalDateTime* 类似。如果需要附加时区功能，可以通过 [TimeZones.jl 包](https://github.com/JuliaTime/TimeZones.jl/) 实现，其汇编了来自 [IANA 时区数据库](http://www.iana.org/time-zones) 的数据。[`Date`](@ref) 和 [`DateTime`](@ref) 都基于 [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) 标准，遵循公历（格里高利历）。
值得注意的是，ISO 8601 标准对公元前的日期需要特别处理。通常来说，公元前的最后一天是公元前 1 年的 12 月 31 日，接下来的一天是公元 1 年的 1 月 1 日，公元 0 年是不存在的。但是，在 ISO 8601 标准中，公元前 1 年被表示为 0 年，即 `0001-01-01` 的前一天是 `0000-12-31`，而 `-0001`（没错，年数为-1）的那一年则实际上是公元前 2 年，`-0002` 则表示公元前 3 年，以此类推。

[^1]:
    The notion of the UT second is actually quite fundamental. There are basically two different notions
    of time generally accepted, one based on the physical rotation of the earth (one full rotation
    = 1 day), the other based on the SI second (a fixed, constant value). These are radically different!
    Think about it, a "UT second", as defined relative to the rotation of the earth, may have a different
    absolute length depending on the day! Anyway, the fact that [`Date`](@ref) and [`DateTime`](@ref)
    are based on UT seconds is a simplifying, yet honest assumption so that things like leap seconds
    and all their complexity can be avoided. This basis of time is formally called [UT](https://en.wikipedia.org/wiki/Universal_Time)
    or UT1. Basing types on the UT second basically means that every minute has 60 seconds and every
    day has 24 hours and leads to more natural calculations when working with calendar dates.

## 构造函数

[`Date`](@ref) 和 [`DateTime`](@ref) 类型可以通过整数或 [`Period`](@ref) 类型，解析，或调整器来构造（稍后会详细介绍）：

In [ ]:
julia> DateTime(2013)
2013-01-01T00:00:00

julia> DateTime(2013,7)
2013-07-01T00:00:00

julia> DateTime(2013,7,1)
2013-07-01T00:00:00

julia> DateTime(2013,7,1,12)
2013-07-01T12:00:00

julia> DateTime(2013,7,1,12,30)
2013-07-01T12:30:00

julia> DateTime(2013,7,1,12,30,59)
2013-07-01T12:30:59

julia> DateTime(2013,7,1,12,30,59,1)
2013-07-01T12:30:59.001

julia> Date(2013)
2013-01-01

julia> Date(2013,7)
2013-07-01

julia> Date(2013,7,1)
2013-07-01

julia> Date(Dates.Year(2013),Dates.Month(7),Dates.Day(1))
2013-07-01

julia> Date(Dates.Month(7),Dates.Year(2013))
2013-07-01

[`Date`](@ref) or [`DateTime`](@ref) parsing is accomplished by the use of format strings. Format
strings work by the notion of defining *delimited* or *fixed-width* "slots" that contain a period
to parse and passing the text to parse and format string to a [`Date`](@ref) or [`DateTime`](@ref)
constructor, of the form `Date("2015-01-01",dateformat"y-m-d")` or
`DateTime("20150101",dateformat"yyyymmdd")`.

有分隔的插入点是通过指定解析器在两个时段之间的分隔符来进行标记的。例如，`"y-m-d"` 会告诉解析器，一个诸如 `"2014-07-16"` 的时间字符串，应该在第一个和第二个插入点之间查找 `-` 字符。`y`，`m` 和 `d` 字符则告诉解析器每一个插入点对应的时段名称。

As in the case of constructors above such as `Date(2013)`, delimited `DateFormat`s allow for
missing parts of dates and times so long as the preceding parts are given. The other parts are given the usual
default values.  For example, `Date("1981-03", dateformat"y-m-d")` returns `1981-03-01`, whilst
`Date("31/12", dateformat"d/m/y")` gives `0001-12-31`.  (Note that the default year is
1 AD/CE.)
Consequently, an empty string will always return `0001-01-01` for `Date`s,
and `0001-01-01T00:00:00.000` for `DateTime`s.

Fixed-width slots are specified by repeating the period character the number of times corresponding
to the width with no delimiter between characters. So `dateformat"yyyymmdd"` would correspond to a date
string like `"20140716"`. The parser distinguishes a fixed-width slot by the absence of a delimiter,
noting the transition `"yyyymm"` from one period character to the next.

Support for text-form month parsing is also supported through the `u` and `U` characters, for
abbreviated and full-length month names, respectively. By default, only English month names are
supported, so `u` corresponds to "Jan", "Feb", "Mar", etc. And `U` corresponds to "January", "February",
"March", etc. Similar to other name=>value mapping functions [`dayname`](@ref) and [`monthname`](@ref),
custom locales can be loaded by passing in the `locale=>Dict{String,Int}` mapping to the `MONTHTOVALUEABBR`
and `MONTHTOVALUE` dicts for abbreviated and full-name month names, respectively.

The above examples used the `dateformat""` string macro. This macro creates a `DateFormat` object once when
the macro is expanded and uses the same `DateFormat` object even if a code snippet is run multiple times.

In [ ]:
julia> for i = 1:10^5
           Date("2015-01-01", dateformat"y-m-d")
       end

Or you can create the DateFormat object explicitly:

In [ ]:
julia> df = DateFormat("y-m-d");

julia> dt = Date("2015-01-01",df)
2015-01-01

julia> dt2 = Date("2015-01-02",df)
2015-01-02

Alternatively, use broadcasting:

In [ ]:
julia> years = ["2015", "2016"];

julia> Date.(years, DateFormat("yyyy"))
2-element Vector{Date}:
 2015-01-01
 2016-01-01

For convenience, you may pass the format string directly (e.g., `Date("2015-01-01","y-m-d")`),
although this form incurs performance costs if you are parsing the same format repeatedly, as
it internally creates a new `DateFormat` object each time.

As well as via the constructors, a `Date` or `DateTime` can be constructed from
strings using the [`parse`](@ref) and [`tryparse`](@ref) functions, but with
an optional third argument of type `DateFormat` specifying the format; for example,
`parse(Date, "06.23.2013", dateformat"m.d.y")`, or
`tryparse(DateTime, "1999-12-31T23:59:59")` which uses the default format.
The notable difference between the functions is that with [`tryparse`](@ref),
an error is not thrown if the string is in an invalid format;
instead `nothing` is returned.  Note however that as with the constructors
above, empty date and time parts assume
default values and consequently an empty string (`""`) is valid
for _any_ `DateFormat`, giving for example a `Date` of `0001-01-01`.  Code
relying on `parse` or `tryparse` for `Date` and `DateTime` parsing should
therefore also check whether parsed strings are empty before using the
result.

A full suite of parsing and formatting tests and examples is available in [`stdlib/Dates/test/io.jl`](https://github.com/JuliaLang/julia/blob/master/stdlib/Dates/test/io.jl).

## Durations/Comparisons

Finding the length of time between two [`Date`](@ref) or [`DateTime`](@ref) is straightforward
given their underlying representation as `UTInstant{Day}` and `UTInstant{Millisecond}`, respectively.
The difference between [`Date`](@ref) is returned in the number of [`Day`](@ref), and [`DateTime`](@ref)
in the number of [`Millisecond`](@ref). Similarly, comparing [`TimeType`](@ref) is a simple matter
of comparing the underlying machine instants (which in turn compares the internal [`Int64`](@ref) values).

In [ ]:
julia> dt = Date(2012,2,29)
2012-02-29

julia> dt2 = Date(2000,2,1)
2000-02-01

julia> dump(dt)
Date
  instant: Dates.UTInstant{Day}
    periods: Day
      value: Int64 734562

julia> dump(dt2)
Date
  instant: Dates.UTInstant{Day}
    periods: Day
      value: Int64 730151

julia> dt > dt2
true

julia> dt != dt2
true

julia> dt + dt2
ERROR: MethodError: no method matching +(::Date, ::Date)
[...]

julia> dt * dt2
ERROR: MethodError: no method matching *(::Date, ::Date)
[...]

julia> dt / dt2
ERROR: MethodError: no method matching /(::Date, ::Date)

julia> dt - dt2
4411 days

julia> dt2 - dt
-4411 days

julia> dt = DateTime(2012,2,29)
2012-02-29T00:00:00

julia> dt2 = DateTime(2000,2,1)
2000-02-01T00:00:00

julia> dt - dt2
381110400000 milliseconds

## Accessor Functions

Because the [`Date`](@ref) and [`DateTime`](@ref) types are stored as single [`Int64`](@ref) values, date
parts or fields can be retrieved through accessor functions. The lowercase accessors return the
field as an integer:

In [ ]:
julia> t = Date(2014, 1, 31)
2014-01-31

julia> Dates.year(t)
2014

julia> Dates.month(t)
1

julia> Dates.week(t)
5

julia> Dates.day(t)
31

While propercase return the same value in the corresponding [`Period`](@ref) type:

In [ ]:
julia> Dates.Year(t)
2014 years

julia> Dates.Day(t)
31 days

Compound methods are provided because it is more efficient to access multiple fields at the same time than individually:

In [ ]:
julia> Dates.yearmonth(t)
(2014, 1)

julia> Dates.monthday(t)
(1, 31)

julia> Dates.yearmonthday(t)
(2014, 1, 31)

One may also access the underlying `UTInstant` or integer value:

In [ ]:
julia> dump(t)
Date
  instant: Dates.UTInstant{Day}
    periods: Day
      value: Int64 735264

julia> t.instant
Dates.UTInstant{Day}(Day(735264))

julia> Dates.value(t)
735264

## Query Functions

Query functions provide calendrical information about a [`TimeType`](@ref). They include information
about the day of the week:

In [ ]:
julia> t = Date(2014, 1, 31)
2014-01-31

julia> Dates.dayofweek(t)
5

julia> Dates.dayname(t)
"Friday"

julia> Dates.dayofweekofmonth(t) # 5th Friday of January
5

Month of the year:

In [ ]:
julia> Dates.monthname(t)
"January"

julia> Dates.daysinmonth(t)
31

As well as information about the [`TimeType`](@ref)'s year and quarter:

In [ ]:
julia> Dates.isleapyear(t)
false

julia> Dates.dayofyear(t)
31

julia> Dates.quarterofyear(t)
1

julia> Dates.dayofquarter(t)
31

The [`dayname`](@ref) and [`monthname`](@ref) methods can also take an optional `locale` keyword
that can be used to return the name of the day or month of the year for other languages/locales.
There are also versions of these functions returning the abbreviated names, namely
[`dayabbr`](@ref) and [`monthabbr`](@ref).
First the mapping is loaded into the `LOCALES` variable:

In [ ]:
julia> french_months = ["janvier", "février", "mars", "avril", "mai", "juin",
                        "juillet", "août", "septembre", "octobre", "novembre", "décembre"];

julia> french_monts_abbrev = ["janv","févr","mars","avril","mai","juin",
                              "juil","août","sept","oct","nov","déc"];

julia> french_days = ["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"];

julia> Dates.LOCALES["french"] = Dates.DateLocale(french_months, french_monts_abbrev, french_days, [""]);

The above mentioned functions can then be used to perform the queries:

In [ ]:
julia> Dates.dayname(t;locale="french")
"vendredi"

julia> Dates.monthname(t;locale="french")
"janvier"

julia> Dates.monthabbr(t;locale="french")
"janv"

Since the abbreviated versions of the days are not loaded, trying to use the
function `dayabbr` will error.

In [ ]:
julia> Dates.dayabbr(t;locale="french")
ERROR: BoundsError: attempt to access 1-element Vector{String} at index [5]
Stacktrace:
[...]

## TimeType-Period Arithmetic

It's good practice when using any language/date framework to be familiar with how date-period
arithmetic is handled as there are some [tricky issues](https://codeblog.jonskeet.uk/2010/12/01/the-joys-of-date-time-arithmetic/)
to deal with (though much less so for day-precision types).

The `Dates` module approach tries to follow the simple principle of trying to change as
little as possible when doing [`Period`](@ref) arithmetic. This approach is also often known as
*calendrical* arithmetic or what you would probably guess if someone were to ask you the same
calculation in a conversation. Why all the fuss about this? Let's take a classic example: add
1 month to January 31st, 2014. What's the answer? Javascript will say [March 3](https://markhneedham.com/blog/2009/01/07/javascript-add-a-month-to-a-date/)
(assumes 31 days). PHP says [March 2](https://stackoverflow.com/questions/5760262/php-adding-months-to-a-date-while-not-exceeding-the-last-day-of-the-month)
(assumes 30 days). The fact is, there is no right answer. In the `Dates` module, it gives
the result of February 28th. How does it figure that out? Consider the classic 7-7-7
gambling game in casinos.

Now just imagine that instead of 7-7-7, the slots are Year-Month-Day, or in our example, 2014-01-31.
When you ask to add 1 month to this date, the month slot is incremented, so now we have 2014-02-31.
Then the day number is checked if it is greater than the last valid day of the new month; if it
is (as in the case above), the day number is adjusted down to the last valid day (28). What are
the ramifications with this approach? Go ahead and add another month to our date, `2014-02-28 + Month(1) == 2014-03-28`.
What? Were you expecting the last day of March? Nope, sorry, remember the 7-7-7 slots. As few
slots as possible are going to change, so we first increment the month slot by 1, 2014-03-28,
and boom, we're done because that's a valid date. On the other hand, if we were to add 2 months
to our original date, 2014-01-31, then we end up with 2014-03-31, as expected. The other ramification
of this approach is a loss in associativity when a specific ordering is forced (i.e. adding things
in different orders results in different outcomes). For example:

In [ ]:
julia> (Date(2014,1,29)+Dates.Day(1)) + Dates.Month(1)
2014-02-28

julia> (Date(2014,1,29)+Dates.Month(1)) + Dates.Day(1)
2014-03-01

What's going on there? In the first line, we're adding 1 day to January 29th, which results in
2014-01-30; then we add 1 month, so we get 2014-02-30, which then adjusts down to 2014-02-28.
In the second example, we add 1 month *first*, where we get 2014-02-29, which adjusts down to
2014-02-28, and *then* add 1 day, which results in 2014-03-01. One design principle that helps
in this case is that, in the presence of multiple Periods, the operations will be ordered by the
Periods' *types*, not their value or positional order; this means `Year` will always be added
first, then `Month`, then `Week`, etc. Hence the following *does* result in associativity and
Just Works:

In [ ]:
julia> Date(2014,1,29) + Dates.Day(1) + Dates.Month(1)
2014-03-01

julia> Date(2014,1,29) + Dates.Month(1) + Dates.Day(1)
2014-03-01

Tricky? Perhaps. What is an innocent `Dates` user to do? The bottom line is to be aware
that explicitly forcing a certain associativity, when dealing with months, may lead to some unexpected
results, but otherwise, everything should work as expected. Thankfully, that's pretty much the
extent of the odd cases in date-period arithmetic when dealing with time in UT (avoiding the "joys"
of dealing with daylight savings, leap seconds, etc.).

As a bonus, all period arithmetic objects work directly with ranges:

In [ ]:
julia> dr = Date(2014,1,29):Day(1):Date(2014,2,3)
Date("2014-01-29"):Day(1):Date("2014-02-03")

julia> collect(dr)
6-element Vector{Date}:
 2014-01-29
 2014-01-30
 2014-01-31
 2014-02-01
 2014-02-02
 2014-02-03

julia> dr = Date(2014,1,29):Dates.Month(1):Date(2014,07,29)
Date("2014-01-29"):Month(1):Date("2014-07-29")

julia> collect(dr)
7-element Vector{Date}:
 2014-01-29
 2014-02-28
 2014-03-29
 2014-04-29
 2014-05-29
 2014-06-29
 2014-07-29

## Adjuster Functions

As convenient as date-period arithmetic is, often the kinds of calculations needed on dates
take on a *calendrical* or *temporal* nature rather than a fixed number of periods. Holidays are
a perfect example; most follow rules such as "Memorial Day = Last Monday of May", or "Thanksgiving
= 4th Thursday of November". These kinds of temporal expressions deal with rules relative to the
calendar, like first or last of the month, next Tuesday, or the first and third Wednesdays, etc.

The `Dates` module provides the *adjuster* API through several convenient methods that
aid in simply and succinctly expressing temporal rules. The first group of adjuster methods deal
with the first and last of weeks, months, quarters, and years. They each take a single [`TimeType`](@ref)
as input and return or *adjust to* the first or last of the desired period relative to the input.

In [ ]:
julia> Dates.firstdayofweek(Date(2014,7,16)) # Adjusts the input to the Monday of the input's week
2014-07-14

julia> Dates.lastdayofmonth(Date(2014,7,16)) # Adjusts to the last day of the input's month
2014-07-31

julia> Dates.lastdayofquarter(Date(2014,7,16)) # Adjusts to the last day of the input's quarter
2014-09-30

The next two higher-order methods, [`tonext`](@ref), and [`toprev`](@ref), generalize working
with temporal expressions by taking a `DateFunction` as first argument, along with a starting
[`TimeType`](@ref). A `DateFunction` is just a function, usually anonymous, that takes a single
[`TimeType`](@ref) as input and returns a [`Bool`](@ref), `true` indicating a satisfied
adjustment criterion.
For example:

In [ ]:
julia> istuesday = x->Dates.dayofweek(x) == Dates.Tuesday; # Returns true if the day of the week of x is Tuesday

julia> Dates.tonext(istuesday, Date(2014,7,13)) # 2014-07-13 is a Sunday
2014-07-15

julia> Dates.tonext(Date(2014,7,13), Dates.Tuesday) # Convenience method provided for day of the week adjustments
2014-07-15

This is useful with the do-block syntax for more complex temporal expressions:

In [ ]:
julia> Dates.tonext(Date(2014,7,13)) do x
           # Return true on the 4th Thursday of November (Thanksgiving)
           Dates.dayofweek(x) == Dates.Thursday &&
           Dates.dayofweekofmonth(x) == 4 &&
           Dates.month(x) == Dates.November
       end
2014-11-27

The [`Base.filter`](@ref) method can be used to obtain all valid dates/moments in a specified
range:

In [ ]:
# Pittsburgh street cleaning; Every 2nd Tuesday from April to November
# Date range from January 1st, 2014 to January 1st, 2015
julia> dr = Dates.Date(2014):Day(1):Dates.Date(2015);

julia> filter(dr) do x
           Dates.dayofweek(x) == Dates.Tue &&
           Dates.April <= Dates.month(x) <= Dates.Nov &&
           Dates.dayofweekofmonth(x) == 2
       end
8-element Vector{Date}:
 2014-04-08
 2014-05-13
 2014-06-10
 2014-07-08
 2014-08-12
 2014-09-09
 2014-10-14
 2014-11-11

Additional examples and tests are available in [`stdlib/Dates/test/adjusters.jl`](https://github.com/JuliaLang/julia/blob/master/stdlib/Dates/test/adjusters.jl).

## Period Types

Periods are a human view of discrete, sometimes irregular durations of time. Consider 1 month;
it could represent, in days, a value of 28, 29, 30, or 31 depending on the year and month context.
Or a year could represent 365 or 366 days in the case of a leap year. [`Period`](@ref) types are
simple [`Int64`](@ref) wrappers and are constructed by wrapping any `Int64` convertible type, i.e. `Year(1)`
or `Month(3.0)`. Arithmetic between [`Period`](@ref) of the same type behave like integers, and
limited `Period-Real` arithmetic is available.  You can extract the underlying integer with
[`Dates.value`](@ref).

In [ ]:
julia> y1 = Dates.Year(1)
1 year

julia> y2 = Dates.Year(2)
2 years

julia> y3 = Dates.Year(10)
10 years

julia> y1 + y2
3 years

julia> div(y3,y2)
5

julia> y3 - y2
8 years

julia> y3 % y2
0 years

julia> div(y3,3) # mirrors integer division
3 years

julia> Dates.value(Dates.Millisecond(10))
10

## Rounding

[`Date`](@ref) and [`DateTime`](@ref) values can be rounded to a specified resolution (e.g., 1
month or 15 minutes) with [`floor`](@ref), [`ceil`](@ref), or [`round`](@ref):

In [ ]:
julia> floor(Date(1985, 8, 16), Dates.Month)
1985-08-01

julia> ceil(DateTime(2013, 2, 13, 0, 31, 20), Dates.Minute(15))
2013-02-13T00:45:00

julia> round(DateTime(2016, 8, 6, 20, 15), Dates.Day)
2016-08-07T00:00:00

Unlike the numeric [`round`](@ref) method, which breaks ties toward the even number by default,
the [`TimeType`](@ref)[`round`](@ref) method uses the `RoundNearestTiesUp` rounding mode. (It's
difficult to guess what breaking ties to nearest "even" [`TimeType`](@ref) would entail.) Further
details on the available `RoundingMode` s can be found in the [API reference](@ref stdlib-dates-api).

Rounding should generally behave as expected, but there are a few cases in which the expected
behaviour is not obvious.

### Rounding Epoch

In many cases, the resolution specified for rounding (e.g., `Dates.Second(30)`) divides evenly
into the next largest period (in this case, `Dates.Minute(1)`). But rounding behaviour in cases
in which this is not true may lead to confusion. What is the expected result of rounding a [`DateTime`](@ref)
to the nearest 10 hours?

In [ ]:
julia> round(DateTime(2016, 7, 17, 11, 55), Dates.Hour(10))
2016-07-17T12:00:00

That may seem confusing, given that the hour (12) is not divisible by 10. The reason that `2016-07-17T12:00:00`
was chosen is that it is 17,676,660 hours after `0000-01-01T00:00:00`, and 17,676,660 is divisible
by 10.

As Julia [`Date`](@ref) and [`DateTime`](@ref) values are represented according to the ISO 8601
standard, `0000-01-01T00:00:00` was chosen as base (or "rounding epoch") from which to begin the
count of days (and milliseconds) used in rounding calculations. (Note that this differs slightly
from Julia's internal representation of [`Date`](@ref) s using Rata Die notation; but since the
ISO 8601 standard is most visible to the end user, `0000-01-01T00:00:00` was chosen as the rounding
epoch instead of the `0000-12-31T00:00:00` used internally to minimize confusion.)

The only exception to the use of `0000-01-01T00:00:00` as the rounding epoch is when rounding
to weeks. Rounding to the nearest week will always return a Monday (the first day of the week
as specified by ISO 8601). For this reason, we use `0000-01-03T00:00:00` (the first day of the
first week of year 0000, as defined by ISO 8601) as the base when rounding to a number of weeks.

Here is a related case in which the expected behaviour is not necessarily obvious: What happens
when we round to the nearest `P(2)`, where `P` is a [`Period`](@ref) type? In some cases (specifically,
when `P <: Dates.TimePeriod`) the answer is clear:

In [ ]:
julia> round(DateTime(2016, 7, 17, 8, 55, 30), Dates.Hour(2))
2016-07-17T08:00:00

julia> round(DateTime(2016, 7, 17, 8, 55, 30), Dates.Minute(2))
2016-07-17T08:56:00

This seems obvious, because two of each of these periods still divides evenly into the next larger
order period. But in the case of two months (which still divides evenly into one year), the answer
may be surprising:

In [ ]:
julia> round(DateTime(2016, 7, 17, 8, 55, 30), Dates.Month(2))
2016-07-01T00:00:00

Why round to the first day in July, even though it is month 7 (an odd number)? The key is that
months are 1-indexed (the first month is assigned 1), unlike hours, minutes, seconds, and milliseconds
(the first of which are assigned 0).

This means that rounding a [`DateTime`](@ref) to an even multiple of seconds, minutes, hours,
or years (because the ISO 8601 specification includes a year zero) will result in a [`DateTime`](@ref)
with an even value in that field, while rounding a [`DateTime`](@ref) to an even multiple of months
will result in the months field having an odd value. Because both months and years may contain
an irregular number of days, whether rounding to an even number of days will result in an even
value in the days field is uncertain.

See the [API reference](@ref stdlib-dates-api) for additional information
on methods exported from the `Dates` module.

# [API reference](@id stdlib-dates-api)

## Dates and Time Types

In [ ]:
Dates.Period
Dates.CompoundPeriod
Dates.Instant
Dates.UTInstant
Dates.TimeType
Dates.DateTime
Dates.Date
Dates.Time
Dates.TimeZone
Dates.UTC

## Dates Functions

In [ ]:
Dates.DateTime(::Int64, ::Int64, ::Int64, ::Int64, ::Int64, ::Int64, ::Int64)
Dates.DateTime(::Dates.Period)
Dates.DateTime(::Function, ::Any...)
Dates.DateTime(::Dates.TimeType)
Dates.DateTime(::AbstractString, ::AbstractString)
Dates.format(::Dates.TimeType, ::AbstractString)
Dates.DateFormat
Dates.@dateformat_str
Dates.DateTime(::AbstractString, ::Dates.DateFormat)
Dates.Date(::Int64, ::Int64, ::Int64)
Dates.Date(::Dates.Period)
Dates.Date(::Function, ::Any, ::Any, ::Any)
Dates.Date(::Dates.TimeType)
Dates.Date(::AbstractString, ::AbstractString)
Dates.Date(::AbstractString, ::Dates.DateFormat)
Dates.Time(::Int64::Int64, ::Int64, ::Int64, ::Int64, ::Int64)
Dates.Time(::Dates.TimePeriod)
Dates.Time(::Function, ::Any...)
Dates.Time(::Dates.DateTime)
Dates.Time(::AbstractString, ::AbstractString)
Dates.Time(::AbstractString, ::Dates.DateFormat)
Dates.now()
Dates.now(::Type{Dates.UTC})
Base.eps(::Union{Type{DateTime}, Type{Date}, Type{Time}, TimeType})

### Accessor Functions

In [ ]:
Dates.year
Dates.month
Dates.week
Dates.day
Dates.hour
Dates.minute
Dates.second
Dates.millisecond
Dates.microsecond
Dates.nanosecond
Dates.Year(::Dates.TimeType)
Dates.Month(::Dates.TimeType)
Dates.Week(::Dates.TimeType)
Dates.Day(::Dates.TimeType)
Dates.Hour(::DateTime)
Dates.Minute(::DateTime)
Dates.Second(::DateTime)
Dates.Millisecond(::DateTime)
Dates.Microsecond(::Dates.Time)
Dates.Nanosecond(::Dates.Time)
Dates.yearmonth
Dates.monthday
Dates.yearmonthday

### Query Functions

In [ ]:
Dates.dayname
Dates.dayabbr
Dates.dayofweek
Dates.dayofmonth
Dates.dayofweekofmonth
Dates.daysofweekinmonth
Dates.monthname
Dates.monthabbr
Dates.daysinmonth
Dates.isleapyear
Dates.dayofyear
Dates.daysinyear
Dates.quarterofyear
Dates.dayofquarter

### Adjuster Functions

In [ ]:
Base.trunc(::Dates.TimeType, ::Type{Dates.Period})
Dates.firstdayofweek
Dates.lastdayofweek
Dates.firstdayofmonth
Dates.lastdayofmonth
Dates.firstdayofyear
Dates.lastdayofyear
Dates.firstdayofquarter
Dates.lastdayofquarter
Dates.tonext(::Dates.TimeType, ::Int)
Dates.toprev(::Dates.TimeType, ::Int)
Dates.tofirst
Dates.tolast
Dates.tonext(::Function, ::Dates.TimeType)
Dates.toprev(::Function, ::Dates.TimeType)

### Periods

In [ ]:
Dates.Period(::Any)
Dates.CompoundPeriod(::Vector{<:Dates.Period})
Dates.value
Dates.default
Dates.periods

### Rounding Functions

`Date` and `DateTime` values can be rounded to a specified resolution (e.g., 1 month or 15 minutes)
with `floor`, `ceil`, or `round`.

In [ ]:
Base.floor(::Dates.TimeType, ::Dates.Period)
Base.ceil(::Dates.TimeType, ::Dates.Period)
Base.round(::Dates.TimeType, ::Dates.Period, ::RoundingMode{:NearestTiesUp})

Most `Period` values can also be rounded to a specified resolution:

In [ ]:
Base.floor(::Dates.ConvertiblePeriod, ::T) where T <: Dates.ConvertiblePeriod
Base.ceil(::Dates.ConvertiblePeriod, ::Dates.ConvertiblePeriod)
Base.round(::Dates.ConvertiblePeriod, ::Dates.ConvertiblePeriod, ::RoundingMode{:NearestTiesUp})

The following functions are not exported:

In [ ]:
Dates.floorceil
Dates.epochdays2date
Dates.epochms2datetime
Dates.date2epochdays
Dates.datetime2epochms

### Conversion Functions

In [ ]:
Dates.today
Dates.unix2datetime
Dates.datetime2unix
Dates.julian2datetime
Dates.datetime2julian
Dates.rata2datetime
Dates.datetime2rata

### Constants

Days of the Week:

| Variable    | Abbr. | Value (Int) |
|:----------- |:----- |:----------- |
| `Monday`    | `Mon` | 1           |
| `Tuesday`   | `Tue` | 2           |
| `Wednesday` | `Wed` | 3           |
| `Thursday`  | `Thu` | 4           |
| `Friday`    | `Fri` | 5           |
| `Saturday`  | `Sat` | 6           |
| `Sunday`    | `Sun` | 7           |

Months of the Year:

| Variable    | Abbr. | Value (Int) |
|:----------- |:----- |:----------- |
| `January`   | `Jan` | 1           |
| `February`  | `Feb` | 2           |
| `March`     | `Mar` | 3           |
| `April`     | `Apr` | 4           |
| `May`       | `May` | 5           |
| `June`      | `Jun` | 6           |
| `July`      | `Jul` | 7           |
| `August`    | `Aug` | 8           |
| `September` | `Sep` | 9           |
| `October`   | `Oct` | 10          |
| `November`  | `Nov` | 11          |
| `December`  | `Dec` | 12          |

#### Common Date Formatters

In [ ]:
ISODateTimeFormat
ISODateFormat
ISOTimeFormat
RFC1123Format

In [ ]:
DocTestSetup = nothing